In [52]:
import json
import pandas as pd
import folium
from geopy.distance import geodesic

### Para obtener las distancias se ha realizado una consulta a la api de [OpenStreetMap](https://overpass-turbo.eu/)

<div style="text-align: center;">
  <a href="https://www.openstreetmap.org/#map=7/53.462/-8.240">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Openstreetmap_logo.svg/1024px-Openstreetmap_logo.svg.png" alt="OpenStreetMap" width="200" height="200">
  </a>
  <a href="https://overpass-turbo.eu/">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Overpass-turbo.svg/1024px-Overpass-turbo.svg.png" alt="Overpass Turbo" width="200" height="200">
  </a>
</div>


In [60]:
house = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip_final.csv', sep = ";")

In [55]:
cinema = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_coordenadas_servicios/cinema.csv')

In [36]:
cinema.columns

Index(['Cinema ', 'longitude', 'latitude'], dtype='object')

In [61]:
house['longitude']

0       41.391564
1       41.395710
2       41.387414
3       41.385963
4       41.388822
          ...    
8913    41.395806
8914    41.401721
8915    41.405616
8916    41.401888
8917    41.406335
Name: longitude, Length: 8918, dtype: float64

In [57]:
cinema['longitude']

0     41.553886
1     41.609813
2     41.401554
3     41.378533
4     41.541105
        ...    
59    41.540933
60    41.607356
61    41.614180
62    41.555052
63    41.555052
Name: longitude, Length: 64, dtype: float64

In [ ]:
def obtener_estacion_mas_cercana(house_row, cinema_df):

    if 'latitude' not in house_row or 'longitude' not in house_row:
        raise ValueError("La fila de la casa no tiene columnas 'latitude' o 'longitude'")

    house_coords = (house_row['latitude'], house_row['longitude'])
    cinema_df['distancia'] = cinema_df.apply(lambda cinema_row: 
                                              geodesic(house_coords, (cinema_row['latitude'], cinema_row['longitude'])).meters, 
                                              axis=1)

    closest_station_row = cinema_df.loc[cinema_df['distancia'].idxmin()]

    estacion_cercana = closest_station_row['Cinema ']
    distancia_minima = closest_station_row['distancia']

    return estacion_cercana, distancia_minima

cinema[['longitude', 'latitude']] = cinema[['longitude', 'latitude']].apply(pd.to_numeric, errors='coerce')

print(cinema.columns)

house[['latitude', 'longitude']] = house[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

house[['cinema', 'distancia_cinema']] = house.apply(
    lambda row: pd.Series(obtener_estacion_mas_cercana(row, cinema)), axis=1
)


Index(['Cinema ', 'longitude', 'latitude'], dtype='object')
      propertyCode                                          thumbnail  \
0         99117414  https://img4.idealista.com/blur/WEB_LISTING-M/...   
1        101300670  https://img4.idealista.com/blur/WEB_LISTING-M/...   
2        104772587  https://img4.idealista.com/blur/WEB_LISTING-M/...   
3        100770574  https://img4.idealista.com/blur/WEB_LISTING-M/...   
4        105932101  https://img4.idealista.com/blur/WEB_LISTING-M/...   
...            ...                                                ...   
8913     105882581  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8914      81411842  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8915     103461663  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8916      98964029  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8917     105093241  https://img4.idealista.com/blur/WEB_LISTING-M/...   

     externalReference  numPhotos    floor       price property

In [64]:
house.to_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_distancias/distancia_cinema.csv',
                       index = False,
                       sep = ";")